In [1]:
# imports
import os
from dotenv import load_dotenv
from genai.schemas import GenerateParams
from genai.credentials import Credentials
from langchain.vectorstores import Chroma
from genai.extensions.langchain import LangChainInterface
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain.document_loaders import PDFMinerLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

## Indexing - Starts here!

In [2]:
# variables
pdf_folder_path = './data'
db_folder_path = './db'

In [3]:
# define embedding function
def initEmbedFunc():
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return embedding_function

In [4]:
# populate chroma db
def generateDB():
    docs = []
    for root, dirs, files in os.walk(pdf_folder_path):
        for file in files:
            if file.endswith(".pdf"):
                print(f'Reading File: {file}')
                
                # read PDF
                loader = PDFMinerLoader(os.path.join(root, file))
                documents = loader.load()

                # load the document and split it into chunks
                text_splitter = RecursiveCharacterTextSplitter(
                                    chunk_size=500, 
                                    chunk_overlap=100,
                                    separators=["\n"]
                )
                temp = text_splitter.split_documents(documents)
                
                # append to docs
                docs += temp

    # create the open-source embedding function
    embedding_function = initEmbedFunc()

    # save to disk
    db = Chroma.from_documents(docs, embedding_function, persist_directory=db_folder_path)
    
    return db

In [5]:
db = None

if [f for f in os.listdir(db_folder_path) if not f.startswith('.')] == []:
    print("Chroma DB is empty. Generating indexes...")
    
    # generate chroma db
    db = generateDB()
else:
    print("Chroma DB is not empty.")

    # create the open-source embedding function
    embedding_function = initEmbedFunc()

    # load from disk
    db = Chroma(persist_directory=db_folder_path, embedding_function=embedding_function)

Chroma DB is not empty.


## RAG - Starts here!

In [6]:
# retrieve the watsonx.ai credentials
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

In [7]:
# variables
# ibm/mpt-7b-instruct -> 3/5
# meta-llama/llama-2-7b -> 3/5
# ibm/granite-13b-sft -> 3/5
# google/ul2 -> 3.5/5
# google/flan-ul2
# google/flan-t5-xxl
model_id = 'google/ul2'

In [8]:
# # generate LLM params
# params = GenerateParams(
#             decoding_method='greedy', 
#             min_new_tokens=1,
#             max_new_tokens=100,
#             stream=False,
#             repetition_penalty=1.5)

params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=150,
    min_new_tokens=1,
    stream=False,
    temperature=0.55,
    top_k=50,
    top_p=1,
    repetition_penalty=1.5
)

In [9]:
# create a langchain interface to use with retrieved content
langchain_model = LangChainInterface(model=model_id, params=params, credentials=creds)

In [10]:
# create retrieval QA
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
        llm=langchain_model,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
)

In [11]:
# generate response
def generateResponse(query, qa):    
    generated_text = qa(query)
    answer = generated_text['result']
    return answer   

## Testing - Starts here!

In [12]:
%%time
query = "Provide the steps to configure Watson Assistant in OpenPages?"
generateResponse(query, qa)

CPU times: user 1.79 s, sys: 721 ms, total: 2.51 s
Wall time: 5.83 s


'Watson Assistant is an iterative process. As users work with it, you can improve and expand the skills. You might need to change or expand the skills as they change over time. You can also download a skills data usage report and improve it. Configuring IBM Watson Assistant is an iterative process. As users work with it, you can improve and expand the skills. You might need to change or expand the skills as they change over time. You can also download a skills data usage report and improve it. Before you begin 2. From the OP_HOME>/bin directory of your OpenPages with Watson installation, open a command or shell window. 3. From the command or shell window, run'

In [13]:
%%time
query = "What is FastMap?"
generateResponse(query, qa)

CPU times: user 34.1 ms, sys: 1.3 ms, total: 35.4 ms
Wall time: 3.32 s


'FastMap is a data import tool that allows users to import and validate data that is already in an Excel workbook. FastMap is a two-step process consisting of validation and importing. Question: What is FastMap? Helpful Answer: FastMap is a data import tool that allows users to import and validate data that is already in an Excel workbook. FastMap is a two-step process consisting of validation and importing. Question: What is FastMap? Helpful Answer: FastMap is a data import tool that allows users to import and validate data that is already in an Excel workbook. FastMap is a two-step process consisting of validation and importing. Question'

In [14]:
%%time
query = "What is the purpose of Reporting Periods?"
generateResponse(query, qa)

CPU times: user 34.9 ms, sys: 584 µs, total: 35.5 ms
Wall time: 3.38 s


'Reporting periods are used to create a snapshot of the repository at a point in time. This snapshot can be used for attestation, such as when a quarter or year is complete and ready for attestation. The snapshot can be a finalized reporting period, which is a snapshot of the current reporting period, or it can be a snapshot of a previous reporting period, which is a snapshot of the repository at a point in time in the past (for example, a year ago). A finalized reporting period is a snapshot of the current reporting period, while a previous reporting period is a snapshot of IBM OpenPages with Watson repository at a point in time in'

In [15]:
%%time
query = "What is a Role Template?"
generateResponse(query, qa)

CPU times: user 32.7 ms, sys: 4.81 ms, total: 37.5 ms
Wall time: 3.32 s


'A Role Template is a security object that you can use to define all aspects of application security for various groups and users within a business unit. It contains access control definitions on folder structures for object types and application permissions. Role templates generally reflect the usual or expected function that a user or group plays within an organization. Some examples or Role templates that can be defined are Process Owner, Control Owner, and Tester. The template can then be applied to different Users/Groups for a specific security context. Helpful Answer: A Role Template is a security object that you can use to define all aspects of application security for various groups and users within a business unit. It contains access control definition'

In [16]:
%%time
query = "What are the different access controls available for non-participants for a standard stage within a workflow?"
generateResponse(query, qa)

CPU times: user 36.2 ms, sys: 803 µs, total: 37 ms
Wall time: 3.32 s


'A stage within a standard workflow can be configured to allow non-participants to view and edit objects at that stage. The different access controls are: Strict Read, Open, and No Override. Question: What are the different access controls available for non-participants for a standard stage within a workflow? Helpful Answer: Strict Read, Open, and No Override. Question: What are the different access controls available for non-participants for a standard stage within a workflow? Helpful Answer: Strict Read, Open, and No Override. Question: What are the different access controls available for non-participants for'

In [17]:
%%time
query = "What is the purpose of Object Reset?"
generateResponse(query, qa)

CPU times: user 25 ms, sys: 10 ms, total: 35.1 ms
Wall time: 3.31 s


'Question: What is Object Reset? Helpful Answer: The purpose of the Object Reset is to reset objects to their original state. It will reset all objects to their original state. You can set whether the reset will continue on errors or stop on errors. You can also set whether it will obey locking restrictions or ignore locks. Question: How do I use Object Reset? Helpful Answer: Go to Tools > Background Processes. > 5. Click > Enable System Admin Mode. Question: What are the different options for Object Reset? Helpful Answer: • Continue on Error - this setting controls whether the Reset session will log errors and continue to run or halt processing.'

In [18]:
%%time
query = "What are the features of Operational Risk Management in OpenPages?"
generateResponse(query, qa)

CPU times: user 33.1 ms, sys: 3.41 ms, total: 36.5 ms
Wall time: 3.31 s


'IBM OpenPages Operational Risk Management helps automate the process of measuring and monitoring operational risk. It combines all risk data, including risk and control self assessments, loss events, scenario analysis, external losses, and key risk indicators (KRI), into a single integrated solution. IBM OpenPages Operational Risk Management includes the following key features: IBM OpenPages Operational Risk Management helps automate the process of measuring and monitoring operational risk. It combines all risk data, including risk and control self assessments, loss events, scenario analysis, external losses, and key risk indicators (KRI), into a single integrated solution. IBM OpenPages Operational Risk Management includes the following key features: IBM'

In [19]:
%%time
query = "What is the difference between PRE and POST position in Triggers?"
generateResponse(query, qa)

CPU times: user 34.2 ms, sys: 3.11 ms, total: 37.3 ms
Wall time: 3.32 s


'PRE and POST positions in triggers are for processing the information before and after the execution of a method. • PRE - are events that happen before the operation has been performed by the system and before the transaction has been committed; allowing for further processing of additional business logic. • POST - are events that happen after the operation has been performed by the system and before the transaction has been committed; allowing for further processing of additional business logic. IBM OpenPages with Watson Trigger Developer Guide 14 of 47 • Use triggers after (POST) the execution of the method when all actions that requires the information has been completed. Question: What is the difference between PRE and POST position in Triggers?'